## Calculating gradients in three ways

Using the example from Chris Olah's [Calculus on Computational Graphs: Backpropagation](http://colah.github.io/posts/2015-08-Backprop/), you'll calcuate gradients for the function `f(a, b) = (a + b) * (b + 1)` in three ways:

1. Numerically (nudge each parameter a little). Simple, fun, slow, a good way to check your work.
2. Analytically (by hand , remembering our rules from calculus). Efficient, error prone.
3. Backprop (reverse mode autodiff). Efficient, automatic.

### Optional exercise
In addition to Chris's article, I recommend this helpful [video](https://www.youtube.com/watch?v=d14TUNcbn1k) from Stanford's cs231 - one of the clearest explanations of backprop on the web (I went through a *ton* of them. If you find something clearer, please let me know so I can share with future students).

As an exercise, try to calculate the gradients for one of the examples in that video using these methods, and verify you can get the same result with each.

In [ ]:
import tensorflow as tf
import numpy as np

## First example

Our first function will be `f(a, b) = (a + b) * (b + 1)`. 

### Forward pass

In [ ]:
def forward(a, b):
  c = a + b # intermediate variables
  d = b + 1
  f = c * d
  return f

In [ ]:
forward(2,1)

### Numeric gradient
Useful to check your work

In [ ]:
def numeric_gradient(my_function, params, h=1e-4):
  
  # Vector of partial derivatives
  grads = []
  
  # Nudge each parameter by h, 
  # calculate how much the function changes
  for i in range(len(params)):
    
    orginal_val = params[i]
    
    # f(x + h)
    params[i] = orginal_val + h
    plus_h = my_function(*params) 
        
    # f(x - h)
    params[i] = orginal_val - h
    minus_h = my_function(*params)
        
    # Partial derivative
    grad = (plus_h - minus_h) / (2 * h)
    grads.append(grad)

    # reset
    params[i] = orginal_val
    
  return grads

In [ ]:
da, db = numeric_gradient(my_function=forward, params=[2, 1])
print ("Numeric gradient. da %0.2f, db %0.2f" % (da, db))

### By hand
Remembering our rules from calculus.

In [ ]:
def by_hand(a, b):
  
  # forward pass, keeping track of intermediate variables
  c = a + b
  d = b + 1
  f = c * d
  
  df = 1 # Gradient of f on itself is 1 (would be replaced by loss)
  dc = d # Product rule
  dd = c # Product rule

  dc_da = 1 # Sum rule
  da = dc * dc_da
  
  dc_db = 1 # Sum rule
  db_path_1 = dc * dc_db
  
  dd_db = 1 # Sum rule
  db_path_2 = dd * dd_db
  
  db = db_path_1 + db_path_2 # Sum over paths (multivariate chain rule)
  
  return da, db

da, db = by_hand(a=2, b=1)
print ("Gradient by backprop. da %0.2f, db %0.2f" % (da, db))

### Autodiff
Reverse mode autodiff using TensorFlow.

Using a GradientTape ([doc](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/GradientTape), [short tutorial](https://www.tensorflow.org/tutorials/customization/autodiff)).

In [ ]:
params = [tf.Variable(2.0), tf.Variable(1.0)]

with tf.GradientTape() as tape:
  result = forward(*params)
grads = tape.gradient(result, params)

print("Autodiff. da %0.2f, db %0.2f" % (grads[0], grads[1]))

## Second example

Here's `f(x, y, z) = (x + y) * z` in the same three ways.

In [ ]:
def forward(x, y, z):
  return (x + y) * z

In [ ]:
forward(x=1, y=2, z=-4)

### Numeric 

In [ ]:
dx, dy, dz = numeric_gradient(my_function=forward, params=[1, 2, -4])
print ("Numeric gradient. dx %0.2f, dy %0.2f, dz %0.2f" % (dx, dy, dz))

### Analytic gradient (by hand)
Backprop by hand, if we rememeber our calculus rules.

In [ ]:
def by_hand(x, y, z):
  
  # Calculate the forward pass, creating intermediate variables as we go.
  q = x + y
  f = q * z
  
  # Propagate the gradient on the function's output 
  # backward through intermediate variables, until we reach the inputs.
  
  # Gradient on the function is just itself. In a model, this would 
  # be replaced by the loss.
  df = 1 
  
  # By product rule.
  dq = z
  
  # By product rule.
  dz = q
  
  dq_dx = 1 # By sum rule
  dq_dy = 1 # By sum rule
  
  # Backprop gradient on q into x
  dx = dq * dq_dx
  
  # Backprop gradient on q into y
  dy = dq * dq_dy
    
  return dx, dy, dz

In [ ]:
dx, dy, dz = by_hand(1., 2., -4.)
print ("By hand. dx %0.2f, dy %0.2f, dz %0.2f" % (dx, dy, dz))

### Autodiff

In [ ]:
params = [tf.Variable(1.0), tf.Variable(2.0), tf.Variable(-4.0)]

with tf.GradientTape() as tape:
  result = forward(*params)
grads = tape.gradient(result, params)

print("Autodiff. dx %0.2f, dy %0.2f dz %0.2f" % 
      (grads[0].numpy(), grads[1].numpy(), grads[2].numpy()))

## A matrix example

`f(W, x) = ReLU(Wx)`

In [ ]:
def forward(W, x):
    wx = tf.matmul(W,x)
    result = tf.nn.relu(wx)
    return result

In [ ]:
W = np.array([[-0.5, 0.2],
              [-0.3, 0.8]])                  
x = np.array([0.2, 0.4])  
x = np.expand_dims(x,1)   
forward(W,x)

### Numeric

In [ ]:
# Updated code to support matmul
def numeric_gradient_v2(my_function, weights, inputs, h=1e-4):
  
  # Partials for each weight
  grads = np.zeros_like(weights)
  
  # Written for clarity, not speed
  for i in range(grads.shape[0]):
    for j in range(grads.shape[1]):
      
      original = weights[i,j]
      
      weights[i,j] = original + h
      plus_h = my_function(weights, inputs)
      
      weights[i,j] = original - h
      minus_h = my_function(weights, inputs)
      
      # partial
      grads[i,j] = np.sum(plus_h - minus_h) / (2 * h)
      
      # reset
      weights[i,j] = original
      
  return grads

In [ ]:
dw = numeric_gradient_v2(forward, weights=W, inputs=x)
print ("dw\n", dw)

### By hand

In [ ]:
def by_hand(W, x):
  q = np.maximum(0, np.matmul(W, x))
  dq = np.ones_like(q)
  dq[q <= 0] = 0  
  dw = np.matmul(dq, x.T)
  return dw

dw = by_hand(W,x)
print ("By hand\n", dw)

### Autodiff

In [ ]:
W = tf.Variable([[-0.5, 0.2],
                 [-0.3, 0.8]])                  
x = tf.Variable([0.2, 0.4])  
x = tf.expand_dims(x,1)   
params = [W, x]
with tf.GradientTape() as tape:
  result = forward(*params)
grads = tape.gradient(result, W)

print("Autodiff. dw", grads)